In [2]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import time

# Datos

In [ ]:
# Load MNIST dataset
# Definimos una secuencia de transformaciones para aplicar a las imágenes del dataset.
# En este caso, solo convertimos las imágenes a tensores utilizando `ToTensor()`.
# Esto es necesario para que las imágenes estén en un formato compatible con PyTorch.
transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])

# Cargamos el dataset MNIST de entrenamiento.
# `root='./data'` especifica el directorio donde se descargarán los datos si no están presentes.
# `train=True` indica que queremos el conjunto de datos de entrenamiento.
# `transform=transform` aplica las transformaciones definidas previamente a cada imagen.
# `download=True` descarga los datos si no están disponibles en el directorio especificado.
mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)

# Creamos un DataLoader que nos permite cargar los datos en lotes pequeños.
# `dataset=mnist_dataset` es el dataset que se cargará.
# `batch_size=16` indica que cada lote contendrá 16 imágenes y etiquetas.
# `shuffle=True` mezcla los datos aleatoriamente en cada época, mejorando la generalización del modelo.
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)

# Obtenemos un único lote de datos del DataLoader.
# `next(iter(data_loader))` convierte el DataLoader en un iterador y toma el primer lote.
# El lote contiene `images` (los tensores de las imágenes) y `labels` (las etiquetas correspondientes).
images, labels = next(iter(data_loader))

#Por alguna razón esto siempre mata mi kernel, por l oque voy a comentarlo
# Plot the images in a grid
#plt.figure(figsize=(10, 10))
#for i in range(16):
 #   plt.subplot(4, 4, i + 1)
  #  plt.imshow(images[i].squeeze(), cmap='gray')
   # plt.title(f'Label: {labels[i].item()}')
    #plt.axis('off')
#plt.tight_layout()
#plt.show()

experimentar neuronas por capa, capas ocultas y función de activación. Se modifica. Solo basta con número de neuronas o capas o la última

# Arquitectura

In [ ]:

# Definimos el modelo MLP
# MLP hereda de nn.Module, lo que permite utilizar las funciones y propiedades de PyTorch
# para crear, entrenar y evaluar redes neuronales.
class MLP(nn.Module):
    def __init__(self):
        # Inicializamos la clase base nn.Module
        # Esto habilita funciones esenciales como la gestión de capas y forward pass.
        super(MLP, self).__init__()
        # Capa completamente conectada: de entrada (28x28 píxeles) a 512 neuronas
        self.fc1 = nn.Linear(28 * 28, 400)
        # Capa oculta: de 512 neuronas a 256 neuronas
        self.fc2 = nn.Linear(400, 200)
        # Capa de salida: de 256 neuronas a 10 clases (números del 0 al 9)
        self.fc3 = nn.Linear(200, 10)
        
        #self.fc4 = nn.Linear(10, 10)
        # Función de activación ReLU
        self.relu = nn.ReLU()
        self.leaky = nn.LeakyReLU()
        self.sigmoid= nn.Sigmoid()

        # Dropout para evitar sobreajuste
        self.dropout = nn.Dropout(0.2)

    # Definimos cómo pasa la información a través de la red
    # Este método es obligatorio en las clases que heredan de nn.Module.
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.relu(self.fc1(x))  # Aplicamos la primera capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.relu(self.fc2(x))  # Aplicamos la segunda capa y ReLU
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.fc3(x)             # Aplicamos la capa de salida
        return x

# Entrenamiento

In [ ]:
# Hiperparámetros
batch_size = 128       # Tamaño de lote
learning_rate = 0.001 # Tasa de aprendizaje
epochs = 7           # Número de épocas de entrenamiento

# Preprocesamiento y carga de datos de MNIST
transform = transforms.Compose([
    transforms.ToTensor(),                 # Convertimos imágenes a tensores
    transforms.Normalize((0.5,), (0.5,))  # Normalizamos a media 0 y varianza 1
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)  # Dataset de entrenamiento
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)  # Dataset de prueba
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)  # Dataloader para entrenamiento
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)  # Dataloader para prueba

# Definimos el modelo, la función de pérdida y el optimizador
model = MLP()                             # Creamos una instancia del modelo MLP
criterion = nn.CrossEntropyLoss()         # Función de pérdida para clasificación
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Optimizador Adam

tiempo_inicial = time.time()

# Bucle de entrenamiento
for epoch in range(epochs):
    model.train()  # Ponemos el modelo en modo entrenamiento
    for images, labels in train_loader:  # Iteramos sobre lotes de datos
        optimizer.zero_grad()            # Reiniciamos los gradientes
        outputs = model(images)          # Hacemos una predicción con el modelo
        loss = criterion(outputs, labels)  # Calculamos la pérdida
        loss.backward()                  # Propagamos los gradientes
        optimizer.step()                 # Actualizamos los pesos del modelo

    # Mostramos la pérdida al final de cada época
    print(f"Época [{epoch+1}/{epochs}], Pérdida: {loss.item():.4f}")

tiempo_final = time.time()

tiempo_transcurrido = tiempo_final - tiempo_inicial

print(f"El tiempo de ejecución del entrenamiento es: {tiempo_transcurrido:.6f} segundos")

Época [1/7], Pérdida: 0.4211
Época [2/7], Pérdida: 0.1642
Época [3/7], Pérdida: 0.0745
Época [4/7], Pérdida: 0.0999
Época [5/7], Pérdida: 0.1624
Época [6/7], Pérdida: 0.0596
Época [7/7], Pérdida: 0.0335
El tiempo de ejecución del entrenamiento es: 42.932278 segundos


# Evaluación del modelo

In [47]:
model.eval()  # Ponemos el modelo en modo evaluación (desactiva Dropout)
correct = 0
total = 0
with torch.no_grad():  # Desactivamos el cálculo de gradientes para evaluación
    for images, labels in test_loader:  # Iteramos sobre los datos de prueba
        outputs = model(images)         # Hacemos predicciones
        _, predicted = torch.max(outputs.data, 1)  # Obtenemos la clase con mayor probabilidad
        total += labels.size(0)         # Total de muestras evaluadas
        correct += (predicted == labels).sum().item()  # Contamos las predicciones correctas

# Calculamos y mostramos la precisión del modelo
accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

Accuracy en el conjunto de prueba: 97.62%


Con la configuración predefinida el resultado de  accuracy es de un 97.73.

1 experimento: modificar el número de neuronas por capa.
#En los comentarios de la clase dice que la primera capa tiene 512, pero realmente son 200.
capa 1:400
capa 2:200
Tiempo ejecución: 78.017469 segundos
Resultado experimento neuronas 1: 97.69

capa1: 600
capa 2:300
Tiempo ejecución: 322.068560 segundos
Resultado experimento neuronas 2: 97.75
Considerando el poco aumento con el aumento de neuronas, usaré el experimento 1 com obase para pasar al siguiente parámetro.

Experimento función activacion 1:
f=Leaky Relu en todas las capas ocultas.

tiempo ejecución: 69.204278 segundos

Resultado experimento funcion activacion 1: 97.51

Experimento funcion activación 2:
Agregamos una sigmoide a la capa de salida.

Tiempo de ejecución: 69.204278 segundos
Resultado experimento funciones de activación 2: 96.29. Es una mala idea agregar la sigmoide.

El relu por defecto funciona bien.

Experimento funciones de activación 3.
Ponemos un relu en primera capa y leaky relu en la segunda.

Tiempo de ejecución: 93.484747 segundos
resultado: 97.40
Si bien mejora el resultado sin la sigmoide, el mejor modelo sigue siendo de 2 capas, 400 y 200 neuronas y con función de activación relu. Si me queda tiempo probaré con otra capa.

Experimentos con el entrenamiento:
Resultado con la estructura elegida en el paso anterior:97.77
tiempo de ejecución: 93.484747 segundos

experimento con algoritmo de optimización: Estandar Adam
prueba:AdamW
tiempo: 93.484747 segundos
resultado:97.33

Prueba con RMSprop.
Tiempo de ejecución: 95.484747 segundos
Resultado:97.38

Con estos dos algoritmos, el mejor es el estandar Adam, por lo que lo conservaré de aquí en más.

Tamaño del lote: Defecto 64.
experimento 1: 48
Tiempo de ejecución: 619.019119 segundos
Resultado: 97.76
 
 Experimento 2: 80
 Tiempo de ejecución:71.038557 segundos
 Resultado: 97.95

Experimento 3: 128
Tiempo de ejecución: 61.631050 segundos
Resultado: 97.84
 
 Experimento 4: 100
 Tiempo ejecución: 65.367842 segundos
 Resultado: 97.57

 Me quedo con el lote de 80, Si bien está por sobre los 10 segundos del modelo más eficiente, alcanza un mayor accuracy con el lote de 80. En este contexto prefiero privilegiar la accuracy, aunque en desafíos más grandes es probable que eligiera al eficiencia.

experimento épocas.

estandar 10:

Prueba 15
Tiempo de ejecución: 91.781324 segundos
Resultado: 97.95


Experimento 7
Tiempo de ejecución: 42.932278 segundos
Resultado:97.62

Si bien el resultado de las 7 épocas es aceptable y demora la mitad del de 15 épocas, creo que las 10 épocas estandar son el equilibrio entre accuracy y tiempo de ejecución.

Probamos la red neuronal

In [3]:
# Verificar si hay una GPU disponible, de lo contrario usar la CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocesamiento: Definir transformaciones para los datos
transform = transforms.Compose([
    transforms.ToTensor(),                # Convertir imágenes a tensores
    transforms.Normalize((0.5,), (0.5,))  # Normalizar los valores a un rango de [-1, 1]
])

# Cargar el conjunto de datos MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)  # Datos de entrenamiento
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)  # Datos de prueba

# Crear DataLoaders para manejar los datos de forma eficiente
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)   # Loader para entrenamiento (batch de 128, mezclado)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)    # Loader para prueba (batch de 128, sin mezclar)

class CNN(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, dropout=0.2, final_layer_size=128):
        super(CNN, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = dropout
        self.final_layer_size = final_layer_size

        # Primera capa convolucional
        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Segunda capa convolucional
        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        # Calcular automáticamente las dimensiones de la capa lineal (fc1)
        self.fc1_input_size = self._calculate_fc1_input_size()
        
        # Primera capa completamente conectada
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)  # Capa de salida para 10 clases (MNIST)

    def _calculate_fc1_input_size(self):
        """
        Calcula automáticamente el tamaño de la entrada para la primera capa completamente conectada (fc1).
        Simula una pasada con una imagen de prueba de tamaño (1, 28, 28).
        """
        with torch.no_grad():  # Desactiva gradientes
            x = torch.randn(1, 1, 28, 28)  # Tensor ficticio de entrada con tamaño MNIST (batch_size=1)
            x = self.pool(torch.relu(self.conv1(x)))  # Aplicar Conv1 -> Pool
            x = self.pool(torch.relu(self.conv2(x)))  # Aplicar Conv2 -> Pool
            fc1_input_size = x.numel()  # Calcular número total de elementos
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")  # Imprime la dimensión de la entrada

        # Primera capa convolucional, ReLU y MaxPooling
        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")  # Dimensión después de Conv1 y Pool

        # Segunda capa convolucional, ReLU y MaxPooling
        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")  # Dimensión después de Conv2 y Pool

        # Aplanar las características 2D a 1D
        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")  # Dimensión después de Flatten

        # Primera capa completamente conectada
        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")  # Dimensión después de fc1

        # Aplicar Dropout
        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")  # Dimensión después de Dropout

        # Capa de salida
        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")  # Dimensión después de fc2 (salida final)

        return x



In [18]:
# Inicializar el modelo, la función de pérdida y el optimizador


model = CNN(verbose=False, filters_l1=16, filters_l2=64, dropout=0.2, final_layer_size=128).to(device)                             # Mover el modelo a la GPU/CPU
criterion = nn.CrossEntropyLoss()                    # Función de pérdida para clasificación multiclase
optimizer = optim.RMSprop(model.parameters(), lr=0.001) # Optimizador Adam con tasa de aprendizaje 0.001

# Definir la función de entrenamiento
def train(model, loader, criterion, optimizer, device):
    model.train()  # Establecer el modelo en modo de entrenamiento
    running_loss = 0.0
    for images, labels in loader:  # Iterar sobre los lotes de datos
        images, labels = images.to(device), labels.to(device)  # Mover los datos a la GPU/CPU

        optimizer.zero_grad()       # Reiniciar los gradientes
        outputs = model(images)     # Paso hacia adelante
        loss = criterion(outputs, labels)  # Calcular la pérdida
        loss.backward()             # Paso hacia atrás (cálculo de gradientes)
        optimizer.step()            # Actualizar los pesos

        running_loss += loss.item()  # Acumular la pérdida
    return running_loss / len(loader)  # Devolver la pérdida promedio

# Definir la función de evaluación
def evaluate(model, loader, device):
    model.eval()  # Establecer el modelo en modo de evaluación
    correct = 0
    total = 0
    with torch.no_grad():  # Deshabilitar el cálculo de gradientes para ahorrar memoria
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)  # Mover datos a la GPU/CPU
            outputs = model(images)  # Paso hacia adelante
            _, predicted = torch.max(outputs, 1)  # Obtener las predicciones (clase con mayor probabilidad)
            total += labels.size(0)  # Contar el número total de ejemplos
            correct += (predicted == labels).sum().item()  # Contar las predicciones correctas
    return correct / total  # Calcular la precisión

# Bucle principal de entrenamiento
num_epochs = 10  # Número de épocas

tiempo_inicial2 = time.time()
for epoch in range(num_epochs):
    # Entrenar el modelo y calcular la pérdida
    train_loss = train(model, train_loader, criterion, optimizer, device)
    # Evaluar el modelo en el conjunto de prueba
    test_accuracy = evaluate(model, test_loader, device)
    # Imprimir los resultados de la época actual
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")



tiempo_final2 = time.time()

tiempo_transcurrido2 = tiempo_final2 - tiempo_inicial2

print(f"El tiempo de ejecución del entrenamiento neuronal es: {tiempo_transcurrido2:.6f} segundos")

# Calcular la precisión final en el conjunto de prueba
final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")


Epoch [1/10], Loss: 0.2114, Test Accuracy: 0.9852
Epoch [2/10], Loss: 0.0589, Test Accuracy: 0.9879
Epoch [3/10], Loss: 0.0399, Test Accuracy: 0.9899
Epoch [4/10], Loss: 0.0292, Test Accuracy: 0.9905
Epoch [5/10], Loss: 0.0225, Test Accuracy: 0.9917
Epoch [6/10], Loss: 0.0190, Test Accuracy: 0.9894
Epoch [7/10], Loss: 0.0158, Test Accuracy: 0.9929
Epoch [8/10], Loss: 0.0129, Test Accuracy: 0.9913
Epoch [9/10], Loss: 0.0112, Test Accuracy: 0.9909
Epoch [10/10], Loss: 0.0090, Test Accuracy: 0.9913
El tiempo de ejecución del entrenamiento neuronal es: 140.757827 segundos
Final Test Accuracy: 0.9913


experimentos red neuronal:

Probamos los valores predefinidos:
Tiempo de ejecución: 125.909052 segundos
Resultado:0.9902

Vamos a empezar con los filtros, subiré a 16 y 64. Por lo que dice la documentación esto no conviene en tareas de este tipo, pero la documentación puede decir lo que uqiera jj.
Prueba (16, 64)
Tiempo de ejecución: 150.085726 segundos
resultado:0.9933

Prueba (32, 128) Creo que es volar muy cerca de lsol, pero veremos que pasa.
Tiempo de ejecución: 243.019903 segundos
Resultado:0.9915
Creo que es obvio que llegamos a un punto muerto. 16 y 64 es lo mejor que podemos conseguir en esto, tanto por tiempo de ejecución como por aumento del accuracy. Aunque por estandar no está mal considerando que demora 30 segundos menos.

experimento capa luego de convolución: Estandar 128
prueba: 192
Tiempo de ejecución: 159.389823 segundos
Resultado: 0.9917
Creo que sesobreajusta por cantidad de neuronas, igual probaré con 256 a ver que pasa.

prueba: 256.
tiempo de ejecución: 165.243595 segundos
Resultado:0.9897
Efectivamente se sobreajusta por muchas neuronas. Probaré con 64 a ver que pasa, pero al parecer 128 es el número.

prueba 64:
Tiempo de ejecución:140.146199 segundos
resultado:0.9918
Parecido a 192, pero con algunos segundos menos en la ejecución. No está mal, pero me quedo con 128.

Experimentos  dropout: Estandar 0.2

prueba 0.4
Tiempo de ejecución: 249.527926 segundos
Resultado:0.9912

Prueba 0.5:
Tiempo de ejecución:144.139868 segundos
Resultado:0.9910
El aumento no es la respuesta. Probaré con 0.3 a ver que pasa. Aunque hay que admitir que este segundo experimento tardó mucho menos que el otro.

Prueba 0.3.
Tiempo de ejecución: 152.109582 segundos
Resultado:0.9920
Tenía que ocurrir un aumento, considerando que las anteriores estuvieron cerca del 0.9910. Solo para terminar voy a probar con 0.1.

Prueba 0.1:
Tiempo de ejecución:146.884606 segundos
Resultado: 0.9920
Me quedo con 0.2. Los demás no están mal, pero 0.2 es mejor.

#Conclusión estructura: la mejor combinación es filtros (16, 64), dropout: 0.2 y capas luego de convolución=128.

Prueba con entrenamiento.

Algoritmo base: Adam.

Prueba Learning rate: estandar: 0.001

Prueba 0.01
Tiempo de ejecución: 157.241732 segundos
Resultado:0.9644
Si subiendo un orden el resultado bajó considerablemente, creo inoficioso probar con otros números. Cambiaré de algoritmo y cerraré.

ALgoritmos:
prueba:AdamW con lr de 0.001
Tiempo de ejecución: 394.762783 segundos
Resultado: 0.9905
Demora mucho y no mejora. Descartado.

Prueba RMSprop

Tiempo ejecución:140.757827 segundos
Resultado:0.9913
Alcanza un rendimiento bastante alto en la mitad de las épocas, pero tampoco supera al lagoritmo Adam. Creo que lo estandar fucniona bien.

Se probaron varios algoritmos y el mejor es Adam con un lr de 0.001.

Síntesis gneeral: para los experimentos, el hiperparámetro más relevante fue aumentar los filtros de la 1 y 2 capa. Eso consigue un modelo cuyo rendimiento es algo mejor que la configuración estandar. En cuanto al tiempo, los resultados suelen estar entre 140 y 160 segundos. La mejor accuracy se logra en 150 segundos, siendo el experimento de los filtros. El resto se mueve por ahí, pero con menro accuracy

Conclusión:

Como puede observarse, la red convolucional tiene mucho mejor rendimiento que el perceptrón. Esto no es de sorprender si consideramos que la documentación señala que la neuronal convolucional es precisamente para utilizar en estos casos, mientras que el perceptrón está pensado en propósitos tabulares y numéricos de otro tipo. AUnque también obtiene resultados bastante buenos, solo que la red convolucional es mejor.
Lo que más modificó el resultado del perceptrón fue el cambio en las neuronas de las cpaas, pasando de 200 a 400 en al primera y de 256 a 200 en la segunda. Mientras que para la red convolucional fueron los filtros, pasando de 8 a 16 en la primera capa y de 32 a 64 en la segunda. El dropout y el número de neuronas en la capa final no fueron determinantes en aumentar el rendimiento.

En síntesis, la mejor elección para problemas de este tipo es la red neuronal convolucional, poniendo especial atención en los filtros de las capas. Finalmente, la red neuronal demora dos veces y media el tiempo que el perceptrón en obtener su mejor resultado, 0.9933 en 150 segundos, mientras que el 97.84 del perceptrón se obtuvo en 61 segundos. Aún con esto sigo prefiriendo la red neuronal.